# 3D Variable Extraction and Aggregation
This notebook provides a robust tool for extracting specific 3D variables (such as wind speed and direction) from PALM (Potsdam Atmospheric Large-Eddy Simulation Model) NetCDF output files. It offers functionality to perform temporal aggregation (e.g., calculating hourly averages) on these variables and then saves the processed, aggregated data into new NetCDF files. This process is essential for reducing data size and preparing variables for further analysis and visualization.

## 1. Import Dependencies
This section imports all necessary Python libraries for numerical operations, NetCDF file handling, interactive widget creation for user input, and basic operating system interactions.

In [1]:
import os

import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset

from IPython.display import display
import ipywidgets as widgets

from utils import palm_variables

## 2. Load Simulation Data
This section defines the file paths for the 3D simulation output NetCDF files (for baseline and scenario 1) and the static driver file. It then loads these files into netCDF4 Dataset objects, making their contents accessible for processing.

In [2]:
# Absolute URLs (paths) of 3D simulation output files
file_xy_1 = r"./Data/3d_surface_output/konstanz_4096x4096_v9_Baseline_av_3d_N03.000-wdir-wspeed.nc"
file_xy_2 = r"./Data/3d_surface_output/konstanz_4096x4096_v9_Scenario_1_av_3d_N03.000-wdir-wspeed.nc"
file_static = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Scenario_1-48hr/INPUT/konstanz_4096x4096_v9_Scenario_1_static_N03"

# Read NetCDF files into Dataset objects in read mode ('r')
dataset_1 = nc.Dataset(file_xy_1, mode='r')
dataset_2 = nc.Dataset(file_xy_2, mode='r')
dataset_3 = nc.Dataset(file_static, mode='r')

# Store datasets in a list for easier iteration
file_xy_list = [file_xy_1, file_xy_2]
dataset_list = [dataset_1, dataset_2]

## 3. Variable Selection
This section allows the user to interactively select a 3D variable from the loaded NetCDF datasets. A dropdown widget is provided for selection, and the chosen variable's description and unit (retrieved from the palm_variables module) are displayed for clear identification.

In [3]:
# Extract variable names from the first dataset where the number of dimensions is greater than 2.
# In PALM 3D output files, these typically represent variables with (time, z, y, x) dimensions.
var_names_palm = [var for var in dataset_1.variables if dataset_1.variables[var].ndim > 2]

# Initialize `test_variable` with the second variable in the list (`var_names_palm[1]`),
# as common 3D variables like 'wspeed' are often at this index in specific output files.
test_variable = var_names_palm[1]

# Create a dropdown widget to allow the user to select the desired 3D variable.
drop_down = widgets.Dropdown(
    options=var_names_palm,         # Populate the dropdown with the extracted 3D variable names.
    value=var_names_palm[1],        # Set the initial selected value in the dropdown.
    description='Select test variable:' # Label displayed next to the dropdown.
)

# Define a handler function that will be called whenever the dropdown's value changes.
def dropdown_handler(change):
    global test_variable  # Declare `test_variable` as global to modify it.
    test_variable = change.new     # Update the global `test_variable` with the newly selected value.
    print(f"Selected variable: {test_variable}") # Print the newly selected variable to the console.

# Attach the `dropdown_handler` function to observe changes in the 'value' property of the dropdown.
drop_down.observe(dropdown_handler, names='value')

# Display the dropdown widget in the notebook output.
display(drop_down)

Dropdown(description='Select test variable:', index=1, options=('wdir', 'wspeed'), value='wspeed')

In [4]:
# Check if a `test_variable` has been selected.
if test_variable:
    # Use the `test_variable` directly to get its info, as 3D variables typically don't have '*' in their names
    # for metadata lookup (unlike some 2D variables like 'ta_2m*_xy').
    var_initial = test_variable
    
    # Retrieve the dictionary of information for the `var_initial` from the `palm_variables` module.
    # `.get()` is used to prevent errors if the key is not found, returning an empty dict instead.
    variable_info = palm_variables.variables_dict.get(var_initial, {})
    
    # Extract the 'unit' from `variable_info`, defaulting to 'No unit available' if the key is missing.
    unit = variable_info.get('unit', 'No unit available')
    # Extract the 'description' from `variable_info`, defaulting to 'No description available' if the key is missing.
    description = variable_info.get('description', 'No description available')
    
    # Print the capitalized description and its unit for clarity.
    print(f"{description.capitalize()}, {unit}")

Magnitude of the horizontal wind vector, m/s


## 4. Temporal Aggregation Function
This section defines a crucial helper function, get_aggregate_time_list. This function is responsible for generating lists of time step indices that correspond to specific aggregation windows. This is essential for calculating moving averages over the simulation's time series data.

In [5]:
def get_aggregate_time_list(total_time_steps, aggregate_time_steps):
    """
    Generates a list of time step ranges for temporal aggregation (e.g., for moving averages).
    Each inner list represents a window of time steps.

    Args:
        total_time_steps (int): The total number of time steps available in the simulation output.
        aggregate_time_steps (int): The desired size of the aggregation window (number of time steps).

    Returns:
        list: A list of lists, where each inner list contains integer time indices
              that fall within a specific aggregation window.
    """
    time_lists = []
    
    for i in range(total_time_steps):
        # Case 1: No aggregation or single time step (window size is 1)
        if aggregate_time_steps <= 1:
            time_list = [i]
        # Case 2: Aggregation with a specified window size
        else:
            # Calculate the half-window size. This is used to define the window around the current time step `i`.
            half_window = aggregate_time_steps // 2
            
            # Determine the start and end indices for the aggregation window.
            # The calculation slightly differs for even vs. odd `aggregate_time_steps` to ensure correct centering.
            if aggregate_time_steps % 2 == 0:
                # For even window sizes, the window is centered around `i` by extending `half_window` backward
                # and `half_window - 1` forward, then including `i`.
                # Example: If aggregate_time_steps=6, half_window=3. For `i=10`, range is [10-3, 10+3] = [7, 13]
                # The window is [7, 8, 9, 10, 11, 12] (6 steps).
                time_list = [j for j in range(i - half_window, i + half_window)]
            else:
                # For odd window sizes, the window is perfectly centered around `i`.
                # Example: If aggregate_time_steps=5, half_window=2. For `i=10`, range is [10-2, 10+2+1] = [8, 13]
                # The window is [8, 9, 10, 11, 12] (5 steps).
                time_list = [j for j in range(i - half_window, i + half_window + 1)]
        
        # Filter `time_list` to ensure all indices are within the valid range of the simulation (0 to `total_time_steps - 1`).
        valid_time_list = [j for j in time_list if 0 <= j < total_time_steps]
        time_lists.append(valid_time_list)
    
    return time_lists

## 5. Prepare Data for Aggregation
This section prepares the necessary data for the aggregation process. It extracts the base filenames from the input simulation files and loads the selected 3D variable data from both dataset_1 and dataset_2. It also sets the aggregate_time_steps parameter (defaulting to 6, representing a 1-hour aggregation for 10-minute time steps) and generates the corresponding time_lists for aggregation.

In [6]:
# Extract the base filename (without path and extension) from the first xy output file.
# This will be used in naming the aggregated output files.
filename_xy_1 = os.path.basename(file_xy_1).split('.')[0]
# Extract the base filename for the second xy output file.
# NOTE: The original code had a typo here, using file_xy_1 for both. Corrected to file_xy_2.
filename_xy_2 = os.path.basename(file_xy_2).split('.')[0]

# Load the actual variable data for the `test_variable` from `dataset_1` and `dataset_2`.
# `test_variable` is determined by the dropdown selection in a previous step.
variable_data_1 = dataset_1[test_variable]
variable_data_2 = dataset_2[test_variable]

# Get the full shape of the 3D variable data (time, z, y, x) from `dataset_1`.
# The first element (`[0]`) gives the total number of time steps.
variable_data_shape = np.shape(dataset_1[test_variable])
total_time_steps = variable_data_shape[0]

# Define the aggregation window size.
# A value of 6 means that each aggregated data point will be an average over 6 consecutive time steps.
# (If original time steps are 10 minutes, 6 steps = 60 minutes = 1 hour).
aggregate_time_steps = 6

# Generate the list of time step ranges for aggregation based on the `total_time_steps` and `aggregate_time_steps`.
# This list (`time_lists`) will guide the averaging process.
time_lists = get_aggregate_time_list(total_time_steps, aggregate_time_steps)

## 6. Perform Aggregation and Save Data
This final section iterates through each of the loaded simulation datasets. For each dataset, it computes the temporal aggregate of the selected 3D variable using the previously defined aggregation windows (time_lists). The aggregated data (which is effectively 2D after averaging along the z-dimension) is then saved into new NetCDF files, organized in a subdirectory named after the aggregation window size.

In [7]:
# Iterate through each dataset in the `dataset_list` (e.g., Baseline and Scenario 1 simulations).
for i, current_dataset in enumerate(dataset_list):
    # Extract the variable data for the `test_variable` from the current dataset.
    variable_data = current_dataset[test_variable]
    
    # Get the total number of time steps for the current variable data.
    current_total_time_steps = np.shape(variable_data)[0]
    
    # Regenerate `time_lists` to ensure it's correct for the current dataset's total time steps,
    # in case datasets have different lengths or `aggregate_time_steps` was changed.
    time_lists = get_aggregate_time_list(current_total_time_steps, aggregate_time_steps)
    
    # Initialize a list to store the aggregated 2D arrays for all time steps.
    variable_data_agg = []
    
    # Loop through each generated time window (`time_list`) to compute the aggregate.
    for j, time_window_indices in enumerate(time_lists):
        values_in_window = []
        
        # For each time index within the current window, extract the 2D slice of the variable.
        # We are specifically selecting the first z-layer (index 0) as this notebook focuses on 2D output.
        for time_idx in time_window_indices:
            # Assuming `variable_data` has dimensions (time, z, y, x).
            values_in_window.append(variable_data[time_idx, 0, :, :])
            
        # Compute the mean of all 2D slices collected in the current window along the time axis (axis=0).
        # This results in a single 2D array representing the aggregated value for that time window.
        variable_data_agg.append(np.mean(values_in_window, axis=0))
        
    # Determine the base filename for the output NetCDF file from the original file path.
    source_filename = os.path.basename(file_xy_list[i]).split('.')[0]
    
    # Construct the `output_filename` by appending the `test_variable` name (with '*' removed if present).
    output_filename = f"{source_filename}_{test_variable.replace('*','')}"
    
    # Define the output directory based on the aggregation time steps and type of variable (e.g., 'wind').
    output_dir = f"./output/_03_agg_{aggregate_time_steps}"
    # Create the output directory if it doesn't already exist. `exist_ok=True` prevents an error if it exists.
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the aggregated data to a new NetCDF file.
    # The file is opened in write mode ("w") and formatted as 'NETCDF4_CLASSIC'.
    with Dataset(os.path.join(output_dir, f"{output_filename}.nc"), "w", format='NETCDF4_CLASSIC') as output_dataset:
        # Create dimensions in the new NetCDF file.
        # `num_time`: number of aggregated time steps.
        # `num_z_index`: fixed to 1 as we are extracting a single z-layer.
        # `num_y`: number of rows in the 2D array.
        # `num_x`: number of columns in the 2D array.
        num_time = len(variable_data_agg)
        num_z_index = 1
        num_y = variable_data_agg[0].shape[0] # Number of rows (y-dimension)
        num_x = variable_data_agg[0].shape[1] # Number of columns (x-dimension)
        
        output_dataset.createDimension('time', num_time)
        output_dataset.createDimension('z', num_z_index)
        output_dataset.createDimension('y', num_y) # Create 'y' dimension
        output_dataset.createDimension('x', num_x) # Create 'x' dimension
        
        # Create the variable in the new NetCDF file.
        # Data type is `np.float32`, dimensions are (time, z, y, x), and a fill value is defined.
        data_var = output_dataset.createVariable(f'{test_variable}', np.float32, ('time', 'z', 'y', 'x'), fill_value=-9999.0)
        
        # Fill the newly created variable with the aggregated 2D data.
        # Each aggregated 2D array is assigned to its corresponding time step and the first z-layer.
        for k, array in enumerate(variable_data_agg):
            data_var[k, 0, :, :] = array # Assign the 2D aggregated array to the NetCDF variable

    print(f"Aggregated data for '{test_variable}' saved to: {os.path.join(output_dir, f'{output_filename}.nc')}")

Aggregated data for 'wspeed' saved to: ./output/_03_agg_6_wind\konstanz_4096x4096_v9_Baseline_av_3d_N03_wspeed.nc
Aggregated data for 'wspeed' saved to: ./output/_03_agg_6_wind\konstanz_4096x4096_v9_Scenario_1_av_3d_N03_wspeed.nc
